In [1]:
import os

os.chdir('/Documents/CBA_Stats')

!pwd

import re

# from cbastats import Scraper
from cbastats import ScraperMongo

sina_scraper = ScraperMongo.SinaScraper(ScraperMongo.SINA_SCHEDULE_BASE_URL,ScraperMongo.ENCODING, ScraperMongo.PARSER, ScraperMongo.HEADERS)

In [7]:
# sina_scraper.scrape_sina(season='20-21')

----------------------------------------
Composing url
----------------------------------------
Scraping Schedule
----------------------------------------
Insert Schedule to CBA_Staging.Schedules
----------------------------------------
Operation complete. Success is not garanteed. Check query to verify.
Cleaning up CBA_Staging.Schedules
----------------------------------------
Pulling schedule from CBA_Staging.Schedules
----------------------------------------


  0%|          | 0/20 [00:00<?, ?it/s]

Scraping game stats
----------------------------------------


100%|██████████| 20/20 [01:03<00:00,  3.19s/it]


Append game stats to CBA_Data.PlayerStatsPerGame
----------------------------------------
Operation complete. Success is not garanteed. Check row count to verify.
Final clean up of the staging schedules
----------------------------------------


# dev for mongo

In [9]:
sina_scraper.scraper_params

{'qleagueid': {'20-21': '206',
  '19-20': '205',
  '18-19': '198',
  '17-18': '189',
  '16-17': '180',
  '15-16': '171',
  '14-15': '158',
  '13-14': '136',
  '12-13': '107',
  '11-12': '83',
  '10-11': '69',
  '09-10': '56',
  '08-09': '44',
  '07-08': '9',
  '06-07': '2',
  '05-06': '1'},
 'qmonth': {'全部': '',
  '11': '11',
  '12': '12',
  '01': '01',
  '02': '02',
  '03': '03',
  '04': '04',
  '05': '05',
  '06': '06',
  '07': '07',
  '08': '08',
  '09': '09',
  '10': '10'},
 'qteamid': {'全部': '',
  '广东': '1',
  '江苏': '2',
  '上海': '4',
  '浙江': '6',
  '福建': '7',
  '深圳': '8',
  '辽宁': '9',
  '北京': '10',
  '新疆': '11',
  '吉林': '12',
  '广州': '13',
  '山西': '14',
  '山东': '15',
  '广厦': '16',
  '天津': '113',
  '青岛': '114',
  '四川': '182',
  '同曦': '368',
  '北控': '369'}}

In [12]:
SEASON = '20-21'

In [11]:
scraped_schedule = sina_scraper.scrape_schedule(season='20-21', month='全部', team='全部')

len(scraped_schedule)

In [47]:
page_content = sina_scraper.get_page_content(composed_url,encoding=sina_scraper.encoding, parser=sina_scraper.parser, headers=sina_scraper.headers)

In [48]:
target_table = page_content.find_all("table")[1]

In [1]:
target_table

NameError: name 'target_table' is not defined

In [50]:
headers = [th.text for th in target_table.find('thead').find_all('th')]

In [51]:
tbody = target_table.find('tbody')

In [52]:
trs = tbody.find_all('tr')

In [105]:
sina_scraper.scraper_params

{'qleagueid': {'20-21': '206',
  '19-20': '205',
  '18-19': '198',
  '17-18': '189',
  '16-17': '180',
  '15-16': '171',
  '14-15': '158',
  '13-14': '136',
  '12-13': '107',
  '11-12': '83',
  '10-11': '69',
  '09-10': '56',
  '08-09': '44',
  '07-08': '9',
  '06-07': '2',
  '05-06': '1'},
 'qmonth': {'全部': '',
  '11': '11',
  '12': '12',
  '01': '01',
  '02': '02',
  '03': '03',
  '04': '04',
  '05': '05',
  '06': '06',
  '07': '07',
  '08': '08',
  '09': '09',
  '10': '10'},
 'qteamid': {'全部': '',
  '广东': '1',
  '江苏': '2',
  '上海': '4',
  '浙江': '6',
  '福建': '7',
  '深圳': '8',
  '辽宁': '9',
  '北京': '10',
  '新疆': '11',
  '吉林': '12',
  '广州': '13',
  '山西': '14',
  '山东': '15',
  '广厦': '16',
  '天津': '113',
  '青岛': '114',
  '四川': '182',
  '同曦': '368',
  '北控': '369'}}

In [99]:
len(rows)

342

In [95]:
rows = []
for tr in trs:
    row = {}
    tds = tr.find_all('td')
    for td in zip(headers,tds):
        header=td[0]
        cell_content=td[1]
        cell_text = str(cell_content.text).strip()
        cell_link = None   
        # 提取link
        if cell_content.find('a',href=True):
                cell_link=cell_content.find('a',href=True)['href'].strip()
        
        # 如果不含link        
        if not cell_link:
            row[header]=cell_text
            continue
        else:
            # there is link
            # 并且表头和单元格内容不一样
            # 比如表头是”比分“，单元格内容是”116：131“，且有链接
            # 那么该cell的数据是{'比分':{'比分':'116:131','url':'https://xxx.xxx.xxx'}}
            if (cell_text!=header):
                cell={}
                cell[header]=cell_text
                cell['url']=cell_link
                row[header]=cell
                continue
            # 如果有link，但标题和单元格文本一样
            # 比如标题是“战报”，单元格文本也是“战报”，则没必要再列一遍“战报”
            
            else:
                row[header]=cell_link
    rows.append(row)
# TODO: add season
# TODO: add teamid etc.

for row in rows:
    row['赛季']=SEASON
    row['GameID_Sina']=re.findall('schedule[/]show[/](\d+)[/]',row['比分']['url'])[0]
    # TODO: fix gameID,主队ID，客队ID issue in stats calculation
    row['主队']['HomeTeamID_Sina']=re.findall('team[/]show[/](\d+)[/]',row['主队']['url'])[0]
    row['客队']['AwayTeamID_Sina']=re.findall('team[/]show[/](\d+)[/]',row['客队']['url'])[0]